In [11]:
import sys
sys.path.insert(1, '../utils')
sys.path.insert(1, '../scripts')
import torch
import numpy as np
import open3d as o3d
import torch.nn as nn
from dataset import RigNetDataset
from models import JointNet
from train_attention import train as train_attention 
from train_jointnet import train as train_jointnet
from visualization_utils import visualize_mesh_graph, visualize_attention_heatmap
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [12]:
path = '../data/ModelResource_RigNetv1_preproccessed/mesh_graphs/val.pkl'
dataset = RigNetDataset(path, num_samples=1, seed=42)

In [13]:
G = dataset[0]

In [14]:
G['attn_mask'].sum() / len(G['attn_mask'])

tensor(0.3028)

In [43]:
G['one_ring'][:, :5]

tensor([[   0,  576,    0,  578,    0],
        [ 358, 1493, 1491, 1493, 1499]])

In [44]:
G['one_ring'][:, :5].T

tensor([[   0,  358],
        [ 576, 1493],
        [   0, 1491],
        [ 578, 1493],
        [   0, 1499]])

In [46]:
G['one_ring'].view(-1, 2)

tensor([[   0,  576],
        [   0,  578],
        [   0,  574],
        ...,
        [2104, 2104],
        [2105, 2105],
        [2106, 2106]])

In [48]:
visualize_mesh_graph(
    vertices=G['vertices'].numpy(),
    edge_list=G['one_ring'].T.numpy(),
    joints_gt=G['joints'].numpy()
)

In [16]:
model = JointNet(train_iters=10, infer_iters=50, edge_dropout=15)
attn_head = model.attn_head

In [17]:
epochs = 1
lr=1e-3 # 1e-6
wd=1e-5
optimizer = torch.optim.AdamW(attn_head.parameters(), lr=lr, weight_decay=wd)

pos_frac = G['attn_mask'].sum() / len(G['attn_mask'])
pos_weight = (1 - pos_frac) /pos_frac 
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

pos_frac, pos_weight

(tensor(0.3028), tensor(2.3025))

In [18]:
train_attention(attn_head, optimizer, dataset, loss_fn=loss_fn,
                val_ds=None, epochs=epochs, logdir='../runs/attention')

Epoch 1/1: 100%|██████████| 1/1 [00:01<00:00,  1.03s/it]

Epoch 1: train loss = 9.6696e-01


In [19]:
attn_pred_probs = nn.functional.sigmoid(attn_head(G['vertices'], G['one_ring'], G['geodesic']))
attn_preds = (attn_pred_probs >= 0.5).long()
attn_preds[:5], attn_pred_probs[:5], attn_pred_probs.mean()

(tensor([[1],
         [1],
         [1],
         [1],
         [1]]),
 tensor([[0.5015],
         [0.5015],
         [0.5015],
         [0.5015],
         [0.5015]], grad_fn=<SliceBackward0>),
 tensor(0.5005, grad_fn=<MeanBackward0>))

In [20]:
accuracy_score(G['attn_mask'], attn_preds)

0.4712861888941623

In [21]:
confusion_matrix(G['attn_mask'], attn_preds, labels=[0, 1])

array([[ 469, 1000],
       [ 114,  524]])

In [22]:
visualize_attention_heatmap(G['vertices'].detach().numpy(), 
                            G['one_ring'].view(-1, 2).detach().numpy(), 
                            attn_pred_probs.detach().numpy(), 
                            joints_gt=G['joints'].detach().numpy())

#### Attn Module TL;DR:
- Use WEIGHTED BCEWithLogitsLoss
- 1e-4 TOO SLOW, and 1e-3 jittery but still continues to converge to local minimum. Should use LRScheduler

In [48]:
epochs = 50
lr=1e-4
wd=1e-5
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

In [63]:
model.is_training = True
train_jointnet(model, optimizer, dataset, epochs=epochs)

Epoch 1/50: 100%|██████████| 1/1 [00:03<00:00,  3.52s/it]


[Epoch 1] train=1.9497e-01 (disp 9.2131e-02, joint 1.0284e-01)


Epoch 2/50: 100%|██████████| 1/1 [00:03<00:00,  3.44s/it]


[Epoch 2] train=1.9445e-01 (disp 9.2178e-02, joint 1.0227e-01)


Epoch 3/50: 100%|██████████| 1/1 [00:03<00:00,  3.72s/it]


...

[Epoch 48] train=1.7341e-01 (disp 8.6273e-02, joint 8.7134e-02)


Epoch 49/50: 100%|██████████| 1/1 [00:03<00:00,  3.39s/it]


[Epoch 49] train=1.7417e-01 (disp 8.6618e-02, joint 8.7549e-02)


Epoch 50/50: 100%|██████████| 1/1 [00:03<00:00,  3.38s/it]

[Epoch 50] train=1.7504e-01 (disp 8.7064e-02, joint 8.7977e-02)


In [64]:
joints_pred = model(G['vertices'], G['one_ring'], G['geodesic'])

# h is not updating!
len(joints_pred), len(G['joints']), model.h

(22,
 18,
 Parameter containing:
 tensor(0.0670, requires_grad=True))

In [67]:
# Displacements
d = model.disp_head(G['vertices'], G['one_ring'], G['geodesic'])
d.mean(axis=0), d.std(axis=0)
q = d + G['vertices']
joints = model(G['vertices'], G['one_ring'], G['geodesic'])

In [68]:
visualize_mesh_graph(
    vertices=G['vertices'].detach().numpy(),
    edge_list=G['one_ring'].view(-1, 2).detach().numpy(),
    joints_gt=G['joints'].detach().numpy(),
    joints_pred=joints.detach().numpy()
)

- Debug only the displacement module 
- Overfit to higher numbers of meshes